# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [5]:
# Write your code below.
from dotenv import load_dotenv
import os

load_dotenv()




True

In [6]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [9]:
import os
from glob import glob

# Write your code below.
price_data_path = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(price_data_path, "**/*.parquet"), recursive = True)




For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [13]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
dd_px['Close_lag_1'] = dd_px['Close'].shift(1)
dd_px['Adj_Close_lag_1'] = dd_px['Adj Close'].shift(1)
dd_px['Returns'] = (dd_px['Close'] / dd_px['Close_lag_1']) - 1
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']
dd_feat = dd_px



+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [16]:
# Write your code below.
pd_feat = dd_feat.compute()
pd_feat['Close_Mean'] = pd_feat['Close'].rolling(10).mean()
pd_feat

# it was not necessary to convert to pandas before calculating the rolling mean
# .rolling().mean() works in Dask
# Dask would have done the calculations in a distributed way, which is more efficient



Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range,Close_Mean
Ticker,,,,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.382839,51.502148,56.464592,48.193848,56.330471,4674353.0,2000,NaN,NaN,NaN,8.270744,NaN
A,2000-01-04 00:00:00+00:00,40.068886,47.567955,49.266811,46.316166,48.730328,4765083.0,2000,51.502148,43.382839,-0.076389,2.950645,NaN
A,2000-01-05 00:00:00+00:00,37.583405,44.617310,47.567955,43.141991,47.389126,5758642.0,2000,47.567955,40.068886,-0.062030,4.425964,NaN
A,2000-01-06 00:00:00+00:00,36.152363,42.918453,44.349072,41.577251,44.080830,2534434.0,2000,44.617310,37.583405,-0.038076,2.771820,NaN
A,2000-01-07 00:00:00+00:00,39.165066,46.494991,47.165592,42.203148,42.247852,2819626.0,2000,42.918453,36.152363,0.083333,4.962444,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTS,2025-01-27 00:00:00+00:00,173.029999,173.029999,173.479996,168.320007,168.320007,2404500.0,2025,168.610001,168.610001,0.026214,5.159988,167.592001
ZTS,2025-01-28 00:00:00+00:00,170.419998,170.419998,174.929993,169.460007,173.250000,2164600.0,2025,173.029999,173.029999,-0.015084,5.469986,168.002000
ZTS,2025-01-29 00:00:00+00:00,170.220001,170.220001,171.190002,169.000000,169.699997,2144200.0,2025,170.419998,170.419998,-0.001174,2.190002,168.583000


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.